<a href="https://colab.research.google.com/github/girinath18/AI_Headshot_RG/blob/headshots/Nokia2API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pyngrok

In [20]:
from pyngrok import ngrok

In [21]:
ngrok.set_auth_token("2ga6KvgSqbJ8a8UcyMG6EC2AeTJ_3tcyVNAHQsQXkzpxUgu9p")
public_url = ngrok.connect(5000).public_url

In [22]:
!git clone https://github.com/girinath18/roop.git

Cloning into 'roop'...
remote: Enumerating objects: 1540, done.
remote: Total 1540 (delta 0), reused 0 (delta 0), pack-reused 1540
Receiving objects: 100% (1540/1540), 97.43 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (917/917), done.


In [ ]:
!pip install flask_ngrok
!pip install onnxruntime
!pip install customtkinter
!pip install torch torchvision
!pip install flask
!pip install pyngrok --upgrade
!pip install --upgrade pyngrok
!pip install -r /content/roop/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring tkinterdnd2-universal: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring onnxruntime: markers 'python_version != "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-coreml: markers 'python_version == "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 50.9 MB/s eta 0:00:00
  Installing build dependencies ... d

# **To changing the directory**

In [ ]:
%cd roop

/content/roop


# **Also working code API script**

In [ ]:
from flask import Flask, request, jsonify
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)


TEMPLATES_FOLDER_PATH = "/content/template/Teacher"

def face_swap(source_image_folder, output_base_folder):
    try:

        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")


        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")


        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")


        for source_image_name in source_images:
            source_image_path = os.path.join(source_image_folder, source_image_name)


            if os.path.isfile(source_image_path):
                logging.info(f"Processing source image: {source_image_name}")

                processed_image_paths = set()

                for target_image_name in template_images:
                    target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)


                    if os.path.isfile(target_image_path):
                        logging.info(f"Processing target image: {target_image_name}")

                        output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                        if output_image_path in processed_image_paths:
                            logging.info(f"Skipping already processed image: {output_image_path}")
                            continue


                        command = (
                            f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                            f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                            f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                            f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                        )
                        logging.info(f"Running command: {command}")

                        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                        stdout, stderr = process.communicate()
                        if process.returncode != 0:
                            logging.error(f"Failed to process target image: {target_image_name}")
                            logging.error(f"Command output: {stderr.decode('utf-8')}")
                        else:
                            logging.info(f"Successfully processed target image: {target_image_name}")
                            logging.info(f"Command output: {stdout.decode('utf-8')}")
                            processed_image_paths.add(output_image_path)


                        if len(processed_image_paths) >= len(template_images):
                            break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:

        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)


        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        output_base_folder = "/content"

        face_swap(source_image_folder, output_base_folder)


        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':

    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Creating the json file for template API**

In [3]:
{
  "categories": {
    "category1": [
      "https://drive.google.com/uc?id=1-T922LMrBkHBQL9OuM8ZMWcW91k19oot",
      "https://drive.google.com/uc?id=1-NTz8uGFjXHOfweygfbZI3UaZ7sBGvtt",
      "https://drive.google.com/uc?id=1-EVrOQOBrrH4cIndzFjGQr58gyPBe1Sh",
      "https://drive.google.com/uc?id=1-Ch2-ANY3_r72jwQ63macpj-Ju-HsmVl",
      "https://drive.google.com/uc?id=1-Ay6QO8Bt2Lr97F8jscb22DiQ-MRhNe9",
      "https://drive.google.com/uc?id=1-ADxhsyYbYtfjsuFtHOJlUM52FlkKTDi"
    ],
    "category2": [
      "https://drive.google.com/uc?id=10HQHbfjV_33xa7WhZeYS4DMuQBqNpw3s",
      "https://drive.google.com/uc?id=109B1r5auRVDKuUJhzn0TrLHhbQf7depB",
      "https://drive.google.com/uc?id=1-sZ8lV6OGo4p1Kp7PPah58cZB0KfOiJ1",
      "https://drive.google.com/uc?id=1-sJ-qYh9gAYHrbujcdBUaKOadLUuFtjj",
      "https://drive.google.com/uc?id=1-poq_gCGlIWoTKRQFXgCGs1t_IAjPH2l",
      "https://drive.google.com/uc?id=1-lmK7Fq_7UpqP_a1gP969MFgggfebKwI",
      "https://drive.google.com/uc?id=1-gNkqHvt_wHnQyTskO-SIdxhlB_vDH_o",
      "https://drive.google.com/uc?id=1-ePjWcUSawXII4El6iiI27p05dZkUjgm",
      "https://drive.google.com/uc?id=10Zu6mPX9G50Bi89CkzBTgaIIOe9jPQaP",
      "https://drive.google.com/uc?id=10S62IrVbLyIa7rDlm0v6bL4hw88iCay-",
      "https://drive.google.com/uc?id=10RazuWfDgbAjUQ8e7LnQhqjvZEzwNWfA",
      "https://drive.google.com/uc?id=10M_UvI7fhyOSz4JuHyrdRx1r2QePNCki",
      "https://drive.google.com/uc?id=10LGIUYQptd6O91ld2KaCw52LdT9VVgf5"
    ]
  }
}


{'categories': {'category1': ['https://drive.google.com/uc?id=1-T922LMrBkHBQL9OuM8ZMWcW91k19oot',
   'https://drive.google.com/uc?id=1-NTz8uGFjXHOfweygfbZI3UaZ7sBGvtt',
   'https://drive.google.com/uc?id=1-EVrOQOBrrH4cIndzFjGQr58gyPBe1Sh',
   'https://drive.google.com/uc?id=1-Ch2-ANY3_r72jwQ63macpj-Ju-HsmVl',
   'https://drive.google.com/uc?id=1-Ay6QO8Bt2Lr97F8jscb22DiQ-MRhNe9',
   'https://drive.google.com/uc?id=1-ADxhsyYbYtfjsuFtHOJlUM52FlkKTDi'],
  'category2': ['https://drive.google.com/uc?id=10HQHbfjV_33xa7WhZeYS4DMuQBqNpw3s',
   'https://drive.google.com/uc?id=109B1r5auRVDKuUJhzn0TrLHhbQf7depB',
   'https://drive.google.com/uc?id=1-sZ8lV6OGo4p1Kp7PPah58cZB0KfOiJ1',
   'https://drive.google.com/uc?id=1-sJ-qYh9gAYHrbujcdBUaKOadLUuFtjj',
   'https://drive.google.com/uc?id=1-poq_gCGlIWoTKRQFXgCGs1t_IAjPH2l',
   'https://drive.google.com/uc?id=1-lmK7Fq_7UpqP_a1gP969MFgggfebKwI',
   'https://drive.google.com/uc?id=1-gNkqHvt_wHnQyTskO-SIdxhlB_vDH_o',
   'https://drive.google.com/uc?id=1

# **creating your Flask app to serve the JSON data**

In [ ]:
!pip install flask flask-ngrok


In [ ]:
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
import json
import time
import requests

app = Flask(__name__)
run_with_ngrok(app)


json_path = '/content/drive/MyDrive/templates/categories.json'
with open(json_path) as f:
    data = json.load(f)

@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify(data)

if __name__ == '__main__':
    try:
        app.run()
    except Exception as e:
        print(f"Error occurred: {e}")

        time.sleep(5)
        app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://932e-34-124-206-196.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/May/2024 05:54:30] "GET /categories HTTP/1.1" 200 -


# **Want to add API to display output images**

In [ ]:
from flask import Flask, request, jsonify
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/template/captain/Female"

def face_swap(source_image_folder, output_base_folder):
    try:
        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        # Select only the first source image
        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    if process.returncode != 0:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output: {stderr.decode('utf-8')}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output: {stdout.decode('utf-8')}")
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        output_base_folder = "/content"

        face_swap(source_image_folder, output_base_folder)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)
